In [98]:
import os
import math
import cv2
import pandas as pd

from tqdm import tqdm
from tools.train_val_test_spliter import split

In [3]:
# To split dataset if already splits folder already exits no need to run it
split()

Splitting the ginen dataset into Train Test=0.4 Validation=0.1
Done


In [46]:
train = pd. read_csv('dataset/train.csv')
test = pd. read_csv('dataset/test.csv')
val = pd. read_csv('dataset/val.csv')

In [47]:
train.head()

,Unnamed: 0,Video_url,Action
0,0,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,punch
1,1,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap
2,2,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,kick
3,3,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,punch
4,4,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap


In [48]:
test.head()

,Unnamed: 0,Video_url,Action
0,0,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,kick
1,1,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap
2,2,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,punch
3,3,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,kick
4,4,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,punch


In [49]:
val.head()

,Unnamed: 0,Video_url,Action
0,0,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap
1,1,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap
2,2,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap
3,3,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap
4,4,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,kick


In [50]:
train.drop('Unnamed: 0',
  axis='columns', inplace=True)
val.drop('Unnamed: 0',
  axis='columns', inplace=True)
test.drop('Unnamed: 0',
  axis='columns', inplace=True)

In [51]:
train.head()

,Video_url,Action
0,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,punch
1,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap
2,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,kick
3,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,punch
4,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap


In [52]:
test.head()

,Video_url,Action
0,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,kick
1,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap
2,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,punch
3,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,kick
4,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,punch


In [53]:
val.head()

,Video_url,Action
0,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap
1,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap
2,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap
3,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap
4,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,kick


In [54]:
print(train.shape)
print(test.shape)
print(val.shape)

(143, 2)
(111, 2)
(27, 2)


In [117]:
def convert_to_frame(data, folder_name):
    directory = 'dataset/' + folder_name
    if not os.path.exists(directory):
        os.makedirs(directory)
    for i in tqdm(range(data.shape[0])):
        video_file = data['Video_url'][i]
        # capturing the video from the given path
        capture = cv2.VideoCapture(video_file) 
        #frame rate
        frame_rate = capture.get(5)
        count = 0
        while(capture.isOpened()):
            #current frame number
            frame_id = capture.get(1) 
            read_correctly, frame = capture.read()
            if not read_correctly:
                break
            if (frame_id % math.floor(frame_rate) == 0):
                # storing the frames in a new folder named train_1
                filename = directory + "/" + video_file.split('/')[-1] + "_frame{}.jpg".format(count)
                count += 1
                cv2.imwrite(filename, frame)
        capture.release()
    print("Successfully Converted")

In [118]:
convert_to_frame(train, 'train')

100%|██████████| 143/143 [00:53<00:00,  2.70it/s]

Successfully Converted
